In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
from src.experiments.common import get_run_from_path
from src.experiments.wrangle import get_multirun_statistics

import altair as alt
from altair import datum, expr
from src.visualization.common import setup_altair
setup_altair()

import plotly.express as px

In [ ]:
multirun = get_run_from_path(
    "/Users/soren/Repositories/Speciale/experiment_results"
    "/cifar/2021-10-28/13-40-50"
    )
tag = "err/val"
parameters = ["inference.lr", "trainer.gradient_clip_algorithm", "trainer.gradient_clip_val"]
metrics = get_multirun_statistics(multirun, parameters, "err/val")

In [ ]:
pd.Series(metrics[tag][-10:].mean(), name="3")

In [ ]:
metrics

In [ ]:
alt.themes

In [ ]:
(
    metrics
    .reset_index()
    .rename(columns=lambda x: x.replace(".", "/"))
    .pipe(alt.Chart)
    .mark_line(clip=True)
    .encode(
        x="step",
        y=tag,
        column="trainer/gradient_clip_val",
        row="trainer/gradient_clip_algorithm",
        color=alt.Color("inference/lr:O", scale=alt.Scale(scheme="plasma"))
    )
)
